<a href="https://colab.research.google.com/github/starshine5150/430-Signature-Assignment-/blob/main/Signature_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import requests
from datetime import datetime
from pyngrok import ngrok

# FRED API key (get yours from https://fred.stlouisfed.org/)
FRED_API_KEY = '2f77223fcddf1574a696fbef78ad8cea'
FRED_BASE_URL = 'https://api.stlouisfed.org/fred/series/observations'

# Available metrics
SERIES_DICT = {
    'GDP': 'GDP',
    'Unemployment Rate': 'UNRATE',
    'Inflation (CPI)': 'CPIAUCSL'
}

# Function to fetch data from FRED
def fetch_fred_data(series_id):
    params = {
        'series_id': series_id,
        'api_key': FRED_API_KEY,
        'file_type': 'json',
        'observation_start': '2010-01-01'
    }
    response = requests.get(FRED_BASE_URL, params=params)
    data = response.json()['observations']
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df = df.dropna()
    return df

# Initialize app
app = JupyterDash(__name__)

# Layout
app.layout = html.Div([
    html.H2("U.S. Economic Indicators Dashboard"),

    html.Label("Select First Indicator:"),
    dcc.Dropdown(
        id='indicator-1',
        options=[{'label': k, 'value': v} for k, v in SERIES_DICT.items()],
        value='GDP'
    ),

    html.Label("Select Second Indicator:"),
    dcc.Dropdown(
        id='indicator-2',
        options=[{'label': k, 'value': v} for k, v in SERIES_DICT.items()],
        value='UNRATE'
    ),

    html.Label("Select Year Range:"),
    dcc.RangeSlider(
        id='year-range',
        min=2010,
        max=datetime.now().year,
        value=[2015, 2023],
        marks={year: str(year) for year in range(2010, datetime.now().year + 1)},
        step=1
    ),

    dcc.Graph(id='comparison-graph')
])

# Callback
@app.callback(
    Output('comparison-graph', 'figure'),
    [Input('indicator-1', 'value'),
     Input('indicator-2', 'value'),
     Input('year-range', 'value')]
)
def update_graph(ind1, ind2, year_range):
    df1 = fetch_fred_data(ind1)
    df2 = fetch_fred_data(ind2)

    df1 = df1[df1['date'].dt.year.between(year_range[0], year_range[1])]
    df2 = df2[df2['date'].dt.year.between(year_range[0], year_range[1])]

    fig = px.line()
    fig.add_scatter(x=df1['date'], y=df1['value'], name=ind1)
    fig.add_scatter(x=df2['date'], y=df2['value'], name=ind2)
    fig.update_layout(title="Economic Indicators Comparison",
                      xaxis_title="Date", yaxis_title="Value")
    return fig

# Run app
app.run(mode='inline')

/usr/local/lib/python3.11/dist-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>